In [59]:
import pandas as pd
import numpy as np

We'll train both a Random Forest Classifier and a K-Nearest Neighbors Classifier using our menopause data. We'll then compare the two models and see which one performs better.

# 📖 Loading codebook values

### ℹ Reading in the data and variables to be used

In [60]:
menopause_data = pd.read_csv('./datasets/visit_dfs.csv', dtype='str')
menopause_data.head()

,HAVEPER,EXPENSI,BROKEBO,AGE,EXERCIS,DNTKNOW,AVCIGDA,NOREASO,SIDEEFF,OSTEOPO,...,DIETNUT,LIKEFEL,COMBIN2,BONES3,PRGNANT,SMOKERE,ESTRNJ1,STOPOTH,BONES2,HOURSPA
0,-1,-1,0,53,-9999,-1,-1,-1,-1,1,...,-9999,-1,-9999,-1,1,1,1,-1,-1,-1
1,-1,-1,0,52,-9999,-1,-1,-1,-1,-1,...,-9999,-1,-9999,-1,1,1,1,-1,-1,-1
2,-1,-1,1,46,-9999,-1,-1,-1,-1,-1,...,-9999,-1,-9999,-1,1,1,1,-1,-1,-1
3,-1,-1,0,50,-9999,-1,-1,-1,-1,-1,...,-9999,-1,-9999,-1,1,1,1,-1,-1,4
4,-1,-1,0,52,-9999,-1,-1,-1,-1,-1,...,-9999,-1,-9999,-1,1,1,1,-1,-1,-1


In [61]:
with open('./lista_variables/variables_selected_final.txt', 'r') as f:
    features = f.read().splitlines()

features

['MENODEP',
 'EXERCIS',
 'DIETNUT',
 'OUTCOME',
 'ESTRNJ2',
 'COMBIN2',
 'ESTROG2',
 'E2AVE',
 'ALCHL24',
 'ESTRDA2',
 'HOURSPA',
 'HEIGHT',
 'ESTRDA1',
 'WEIGHT',
 'HAPPY',
 'PRGNANT',
 'VAGINDR',
 'SMOKERE',
 'HOTFLAS',
 'PHYSILL',
 'AVCIGDA',
 'TRBLSLE',
 'CANCERS',
 'DIABETE',
 'BROKEBO',
 'OSTEOPR',
 'STATUS',
 'ESTROG1',
 'ESTRNJ1',
 'AGE',
 'RACE',
 'NUMHOTF',
 'VISIT']

> Do note that our variables are mainly categorical, but not all of them. We'll read a JSON file to know which variables aren't categorical and we'll change them to float.

In [62]:
import json

with open('./lista_variables/variables_selected_labels.json', 'r') as file:
    features_labels = json.load(file)

### ✅ Parsing data to correct data type

In [63]:
integer_features = features_labels['NUMERICAL']['vars']['int']
float_features = features_labels['NUMERICAL']['vars']['float']

for feature in integer_features:
    menopause_data[feature] = menopause_data[feature].astype(int)

for feature in float_features:
    menopause_data[feature] = menopause_data[feature].astype(float)

In [64]:
menopause_data.dtypes

HAVEPER    object
EXPENSI    object
BROKEBO     int32
AGE         int32
EXERCIS    object
            ...  
SMOKERE    object
ESTRNJ1    object
STOPOTH    object
BONES2     object
HOURSPA    object
Length: 64, dtype: object

### ✨ Pipeline for negative values in data

In some of the variables, negative values are used to indicate missing data, or that the patient refused to answer. We'll create a pipeline to replace these negative values with NaNs,
and we'll use KNN to impute the missing values.

This will aid in standardizing the data.

In [65]:
numerical_features = integer_features + float_features
numerical_features.remove('VISIT')


In [66]:
# Set negatives as NaN
menopause_data[numerical_features] = menopause_data[numerical_features].apply(lambda x: x.where(x > 0, np.nan))


from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)

# DONE
# menopause_data[numerical_features] = imputer.fit_transform(menopause_data[numerical_features])

In [67]:
menopause_data = pd.read_csv('./datasets/visit_dfs_imputed.csv')

In [69]:
menopause_data[numerical_features].isna().sum()

AVCIGDA    0
BROKEBO    0
AGE        0
NUMHOTF    0
E2AVE      0
HEIGHT     0
WEIGHT     0
dtype: int64

In [70]:
menopause_data[numerical_features].head()

,AVCIGDA,BROKEBO,AGE,NUMHOTF,E2AVE,HEIGHT,WEIGHT
0,12.4,1.0,53.0,2.8,70.15,155.4,74.0
1,20.4,1.0,52.0,2.8,194.55,161.3,53.2
2,12.2,1.0,46.0,1.8,250.65,166.1,85.6
3,17.8,1.0,50.0,2.8,187.95,161.5,78.4
4,13.2,1.0,52.0,2.2,119.50,149.5,68.0


In [71]:
# Save dataset
# DONE
# menopause_data.to_csv('./datasets/visit_dfs_imputed.csv', index=True)

# 🧠 Model training!

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from mlflow import log_metric, log_param, log_artifact, start_run


In [ ]:
targets = [
    'STATUS',    # Indicating if the patient is menopausal or not, among other options
    'NUMHOTF',  # Number of hot flashes daily
    'OSTEOPR',  # Indicating if the patient has osteoporosis or not
    'MENODEP',  # Indicating if the patient has depression or not
]

In [ ]:
# Shuffle order to avoid order bias
menopause_data = shuffle(menopause_data)


In [ ]:
# Standardize numerical features
scaler = StandardScaler()
# menopause_data[numerical_features] = scaler.fit_transform(menopause_data[numerical_features])
menopause_data[numerical_features].head()

In [ ]:
# Divide data into train and test, but only for the KNN model,
# since RF does the bootstraping datasaet itself

""" X_train, X_test, y_train, y_test = train_test_split(
    
) """

